# PowerShell: Microsoft Graph / User Accounts

### Install and Load the Module

In [ ]:
if (!(Get-Module Microsoft.Graph.Users -ListAvailable)) {
	Install-Module Microsoft.Graph.Users
} else {
	Write-Host "module already installed"
}
Import-Module Microsoft.Graph.Users

### Connect to Graph

In [ ]:
if (!(Get-MgContext)) {
	#Connect-MgGraph -Identity  ## use this in Azure if you have a Managed Identity
	Connect-MgGraph User.Read.All,Organization.Read.All,Directory.Read.All
}

### Get all user accounts

In [ ]:
$users = Get-MgUser -All
$users.Count

### Get properties for one user account

In [ ]:
$users[0] | Select-Object * | Format-List

### Get all user accounts with extended properties

In [ ]:
$UserProps = @('Id','AccountEnabled','DisplayName','Surname','jobTitle','EmployeeId','EmployeeType',
	'userPrincipalName','mail','CompanyName','Department','OfficeLocation','OtherMails','UsageLocation')

$users = $(Get-MgUser -Property $Properties -All -ConsistencyLevel 'eventual' | Select-Object $properties)
$users.Count

In [ ]:
$users[0] | Select-Object * | Format-List

### Disconnect

In [ ]:
if (Get-MgContext) {
	$null = Disconnect-MgGraph
	Write-Host "You are now disconnected"
}

### Combining into a function

In [11]:
function Get-AadUsers {
	[CmdletBinding()]
	param (
		[parameter()][switch]$Identity ## use this in Azure if you have a Managed Identity
	)
	try {
		if (!(Get-MgContext)) {
			if ($Identity) {
				Connect-MgGraph -Identity
			} else {
				Connect-MgGraph User.Read.All,Organization.Read.All,Directory.Read.All
			}
			$internalConnect = $True
		}
		$UserProps = @('Id','AccountEnabled','DisplayName','Surname','jobTitle','EmployeeId','EmployeeType',
			'userPrincipalName','mail','CompanyName','Department','OfficeLocation','OtherMails','UsageLocation')
		$users = $(Get-MgUser -Property $Properties -All -ConsistencyLevel 'eventual' | Select-Object $properties)
		$result = [pscustomobject]@{
			Status = 'Success'
			Users = $users
		}
	}
	catch {
		$result = [pscustomobject]@{
			Status = 'Error'
			Message = $($_.Exception.Message)
		}
	}
	finally {
		Write-Output $result
		if (($internalConnect = $True) -and (Get-MgContext)) {
			$null = Disconnect-MgGraph
		}
	}
}

$response = Get-AadUsers
$response.Users[5]


Id                                   DisplayName Mail                      UserPrincipalName
--                                   ----------- ----                      -----------------       
9b21ee07-a94c-4267-ab46-9459ce8585d6 David Stein david.stein@quisitive.com david.stein_quisitive.c…

